In [74]:
import requests
# https://www.dataquest.io/blog/web-scraping-tutorial-python/

In [2]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [3]:
page.status_code

200

In [5]:
page.content


b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

In [6]:
# We can use the BeautifulSoup library to parse this document

In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [9]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [10]:
list(soup.children)

['html', '\n', <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [11]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [12]:
html = list(soup.children)[2]

In [14]:
list(html.children)

['\n', <head>
 <title>A simple example page</title>
 </head>, '\n', <body>
 <p>Here is some simple content for this page.</p>
 </body>, '\n']

In [24]:
ele_1 = list(html.children)[0]
ele_2 = list(html.children)[1]
ele_3 = list(html.children)[2]
ele_4 = list(html.children)[3]

In [21]:
print(ele_1)

In [22]:
print(ele_2)

<head>
<title>A simple example page</title>
</head>


In [23]:
print(ele_3)

In [25]:
print(ele_4)

<body>
<p>Here is some simple content for this page.</p>
</body>


In [26]:
body = list(html.children)[3]

In [27]:
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [28]:
p = list(body.children)[1]

In [29]:
p.get_text()

'Here is some simple content for this page.'

## Finding all instances of a  tage at once

In [31]:
# use the find_all method, which will find all
# the instances of a tag on a page

In [32]:
soup = BeautifulSoup(page.content,'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [36]:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [37]:
soup.find('p')

<p>Here is some simple content for this page.</p>

### Searching for tags by class and id

In [40]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [42]:
soup.find_all('p',class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [43]:
soup.find_all(class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [44]:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

In [45]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>, <p class="inner-text">
                 Second paragraph.
             </p>]

## Downloading weather data

We now know enough to download the page and start parsing it. In the below code, we:

1. Download the web page containing the forecast.
2. Create a BeautifulSoup class to parse the page.
3. Find the div with id seven-day-forecast, and assign to seven_day
4. Inside seven_day, find each individual forecast item.
5. Extract and print the first forecast item.


In [52]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168#.W2iCp9hKjK0")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day=soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="forecast-tombstone")
today = forecast_items[0]
print(today.prettify())



<li class="forecast-tombstone">
 <div class="tombstone-container">
  <p class="period-name">
   Today
   <br/>
   <br/>
  </p>
  <p>
   <img alt="Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. "/>
  </p>
  <p class="short-desc">
   Sunny
  </p>
  <p class="temp temp-high">
   High: 69 °F
  </p>
 </div>
</li>



### Exacting information from the page

In [56]:
period = today.find(class_="period-name").get_text()
short_desc = today.find(class_="short-desc").get_text()
temp = today.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Today
Sunny
High: 69 °F


In [60]:
img = today.find("img")
# print(img)
desc = img['title']
print(desc)

Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. 


## Exacting all the information from the pape
Combine css selectors and list comprehensions to extract everything at once.

In [63]:
period_tags = seven_day.select(".tombstone-container .period-name")
# where write like .XXX?
periods = [pt.get_text() for pt in period_tags]
periods

['Today',
 'Tonight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday']

In [64]:
# Apply the same techinique to get the other 3 fields
short_descs = [sd.get_text() for sd in seven_day.select(" .tombstone-container  .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny']
['High: 69 °F', 'Low: 55 °F', 'High: 67 °F', 'Low: 54 °F', 'High: 68 °F', 'Low: 55 °F', 'High: 68 °F', 'Low: 56 °F', 'High: 70 °F']
['Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. ', 'Tonight: Partly cloudy, with a low around 55. West southwest wind 15 to 21 mph, with gusts as high as 25 mph. ', 'Tuesday: Mostly sunny, with a high near 67. West southwest wind 11 to 20 mph, with gusts as high as 24 mph. ', 'Tuesday Night: Partly cloudy, with a low around 54. West southwest wind 16 to 18 mph, with gusts as high as 23 mph. ', 'Wednesday: Mostly sunny, with a high near 68. West southwest wind 7 to 17 mph, with gusts as high as 21 mph. ', 'Wednesday Night: Partly cloudy, with a low around 55.', 'Thursday: Mostly sunny, with a high near 68.', 'Thursday Night: Partly cloudy, with a low around 56.', 'Friday: Mos

## Combining our data into a Pandas Dataframe
A DataFrame is an object that can store tabular data, making data analysis easy. In order to combine data into a dataframe, we'll call the DataFrame class, and pass in each list of items that we have. We pass them in as part of a dictionary. Each dictionary key will become a column in the DataFrame, and each list will become the values in the column:

In [66]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc":short_descs,
    "temp":temps,
    "desc":descs
})
weather

,desc,period,short_desc,temp
0,"Today: Sunny, with a high near 69. West wind 1...",Today,Sunny,High: 69 °F
1,"Tonight: Partly cloudy, with a low around 55. ...",Tonight,Partly Cloudy,Low: 55 °F
2,"Tuesday: Mostly sunny, with a high near 67. We...",Tuesday,Mostly Sunny,High: 67 °F
3,"Tuesday Night: Partly cloudy, with a low aroun...",TuesdayNight,Partly Cloudy,Low: 54 °F
4,"Wednesday: Mostly sunny, with a high near 68. ...",Wednesday,Mostly Sunny,High: 68 °F
5,"Wednesday Night: Partly cloudy, with a low aro...",WednesdayNight,Partly Cloudy,Low: 55 °F
6,"Thursday: Mostly sunny, with a high near 68.",Thursday,Mostly Sunny,High: 68 °F
7,"Thursday Night: Partly cloudy, with a low arou...",ThursdayNight,Partly Cloudy,Low: 56 °F
8,"Friday: Mostly sunny, with a high near 70.",Friday,Mostly Sunny,High: 70 °F


We can now do some analysis on the data. For example, we can use a regular expression and the Series.str.extract method to pull out the numeric temperature values:

In [71]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)",expand=False)
weather["temp_num"]=temp_nums.astype('int')


In [70]:
weather["temp_num"].mean()

62.44444444444444

In [72]:
# select rows that happen at night
is_night = weather["temp"].str.contains("Low")
weather["is_night"]=is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [73]:
weather[is_night]

,desc,period,short_desc,temp,temp_num,is_night
1,"Tonight: Partly cloudy, with a low around 55. ...",Tonight,Partly Cloudy,Low: 55 °F,55,True
3,"Tuesday Night: Partly cloudy, with a low aroun...",TuesdayNight,Partly Cloudy,Low: 54 °F,54,True
5,"Wednesday Night: Partly cloudy, with a low aro...",WednesdayNight,Partly Cloudy,Low: 55 °F,55,True
7,"Thursday Night: Partly cloudy, with a low arou...",ThursdayNight,Partly Cloudy,Low: 56 °F,56,True
